In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from env_vars import sentinel_username,sentinel_password
import glob
import pandas as pd
import subprocess

In [2]:
def get_api():
    
    return SentinelAPI(sentinel_username, sentinel_password, "https://scihub.copernicus.eu/apihub/")


def get_products(api, footprint, date_start, date_end,
                 area='IsWithin', raw='1C',
                 platform='Sentinel-2', cloudcover=(0,10)):
    
    return api.query(footprint,
                     date=(date_start, date_end),
                     area_relation=area,
                     raw=raw,
                     platformname=platform,
                     cloudcoverpercentage=cloudcover)


def find_ee_index_matches(api, products, ee_index):
    
    products_df = api.to_dataframe(products)   
    products_df = products_df.reset_index()
    products_df = products_df.rename(columns={'index': 'sentinel_id'})
    products_df = products_df.drop_duplicates(subset=['tileid'])
    
    ee_index_2 = ee_index.reset_index()
    merged = products_df.merge(ee_index_2, left_on='title', right_on='PRODUCT_ID')
    rows = merged['index'].tolist()
            
    return rows


def generate_tci_uri(ee_index, row):
    uri = ee_index.loc[row, 'BASE_URL']
    uri += '/GRANULE/'
    granule_id = ee_index.loc[row, 'GRANULE_ID']
    uri += granule_id
    uri += '/IMG_DATA/'
    tile_id = granule_id.split('_')[1]
    date = ee_index.loc[row, 'DATATAKE_IDENTIFIER'].split('_')[1]
    uri += f'{tile_id}_{date}_TCI.jp2'
    
    return uri


def download_tcis(ee_index, rows, year):
    
    cloud_env = r"C:\Users\David\AppData\Local\Google\Cloud SDK\cloud_env.bat"
    
    for row in rows:
        uri = generate_tci_uri(ee_index, row)
        subprocess.run([cloud_env, '&&', 'gsutil', 'cp', uri, f'D:/canopy_data/{year}'])
        
        
def get_tcis_for_year(year, footprint, ee_index):
    
    api = get_api()
    print('getting products')
    if year < 2015:
        print('ERROR: Year must be 2014 or above')
        return None
    elif year == 2015:
        products = get_products(api, footprint, date(year, 7, 1), date(year, 12, 31))
    else:
        products = get_products(api, footprint, date(year, 1, 1), date(year, 12, 31))
    print('finding rows')
    rows = find_ee_index_matches(api, products, ee_index)
    print('downloading tcis')
    download_tcis(ee_index, rows, year)

In [3]:
footprint = geojson_to_wkt(read_geojson('./data/Geometry/congo_basin_boundary/congo_basin_boundary_custom_v3.geojson'))

In [4]:
ee_index = pd.read_csv('earth-engine-index.csv')

In [12]:
api = get_api()
products = get_products(api, footprint, date(2015, 7, 1), date(2015, 7, 31))
products_df = api.to_dataframe(products)
products_df = products_df.reset_index()
products_df.head()

,index,title,link,link_alternative,link_icon,summary,datatakesensingstart,beginposition,endposition,ingestiondate,...,platformname,size,tileid,hv_order_tileid,filename,identifier,uuid,level1cpdiidentifier,granuleidentifier,datastripidentifier
0,55797ce2-d2a1-4e15-826c-75a4f421a21a,S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:23:28.140,...,Sentinel-2,152.40 MB,35NPA,NA35P,S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_2...,S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_2...,55797ce2-d2a1-4e15-826c-75a4f421a21a,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_DS_EPA__20160807T083950_S2015...
1,1bce998b-c4f2-42d3-b2a1-de8db9187439,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPV_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:16:31.232,...,Sentinel-2,307.36 MB,35MPV,MV35P,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPV_2...,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPV_2...,1bce998b-c4f2-42d3-b2a1-de8db9187439,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_DS_EPA__20160807T083950_S2015...
2,d3af0e3f-24ee-4104-b5b7-7f90e69781b7,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:02:59.677,...,Sentinel-2,469.73 MB,35MPU,MU35P,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPU_2...,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPU_2...,d3af0e3f-24ee-4104-b5b7-7f90e69781b7,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_DS_EPA__20160807T083950_S2015...
3,6ba536c1-41ad-4c15-90ca-6e78c3fe10b7,S2A_MSIL1C_20150708T081016_N0204_R078_T35NQB_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 19:58:05.495,...,Sentinel-2,689.62 MB,35NQB,NB35Q,S2A_MSIL1C_20150708T081016_N0204_R078_T35NQB_2...,S2A_MSIL1C_20150708T081016_N0204_R078_T35NQB_2...,6ba536c1-41ad-4c15-90ca-6e78c3fe10b7,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_TL_EPA__20160807T083950_A0002...,S2A_OPER_MSI_L1C_DS_EPA__20160807T083950_S2015...
4,d89f7c7b-7e9e-4df2-a946-b6804920ba69,S2A_MSIL1C_20150727T085946_N0204_R064_T34NHF_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-27T08:59:46.027Z, Instrument: MS...",2015-07-27 08:59:46.027,2015-07-27 08:59:46.027,2015-07-27 08:59:46.027,2019-02-04 18:18:31.878,...,Sentinel-2,129.71 MB,34NHF,NF34H,S2A_MSIL1C_20150727T085946_N0204_R064_T34NHF_2...,S2A_MSIL1C_20150727T085946_N0204_R064_T34NHF_2...,d89f7c7b-7e9e-4df2-a946-b6804920ba69,S2A_OPER_MSI_L1C_TL_EPA__20160817T102829_A0004...,S2A_OPER_MSI_L1C_TL_EPA__20160817T102829_A0004...,S2A_OPER_MSI_L1C_DS_EPA__20160817T102829_S2015...


In [13]:
ee_index_2 = ee_index.reset_index()

products_df = products_df.rename(columns={'index': 'sentinel_id'})
merged = products_df.merge(ee_index_2, left_on='title', right_on='PRODUCT_ID')
merged.head()

,sentinel_id,title,link,link_alternative,link_icon,summary,datatakesensingstart,beginposition,endposition,ingestiondate,...,SENSING_TIME,TOTAL_SIZE,CLOUD_COVER,GEOMETRIC_QUALITY_FLAG,GENERATION_TIME,NORTH_LAT,SOUTH_LAT,WEST_LON,EAST_LON,BASE_URL
0,55797ce2-d2a1-4e15-826c-75a4f421a21a,S2A_MSIL1C_20150708T081016_N0204_R078_T35NPA_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:23:28.140,...,2015-07-08T08:21:58.000000Z,159641454.0,1.3745,PASSED,2015-07-08T08:21:58.000000Z,0.904542,-0.088357,28.602039,28.885221,gs://gcp-public-data-sentinel-2/tiles/35/N/PA/...
1,1bce998b-c4f2-42d3-b2a1-de8db9187439,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPV_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:16:31.232,...,2015-07-08T08:21:58.000000Z,322125342.0,2.9824,PASSED,2015-07-08T08:21:58.000000Z,0.000090,-0.993003,28.402809,28.885269,gs://gcp-public-data-sentinel-2/tiles/35/M/PV/...
2,d3af0e3f-24ee-4104-b5b7-7f90e69781b7,S2A_MSIL1C_20150708T081016_N0204_R078_T35MPU_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 20:02:59.677,...,2015-07-08T08:21:58.000000Z,492387691.0,5.0505,PASSED,2015-07-08T08:21:58.000000Z,-0.903786,-1.897246,28.202183,28.886014,gs://gcp-public-data-sentinel-2/tiles/35/M/PU/...
3,6ba536c1-41ad-4c15-90ca-6e78c3fe10b7,S2A_MSIL1C_20150708T081016_N0204_R078_T35NQB_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-08T08:10:16.027Z, Instrument: MS...",2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2015-07-08 08:10:16.027,2019-02-04 19:58:05.495,...,2015-07-08T08:21:58.000000Z,722956185.0,0.6652,PASSED,2015-07-08T08:21:58.000000Z,1.808778,0.815551,28.800827,29.783921,gs://gcp-public-data-sentinel-2/tiles/35/N/QB/...
4,d89f7c7b-7e9e-4df2-a946-b6804920ba69,S2A_MSIL1C_20150727T085946_N0204_R064_T34NHF_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2015-07-27T08:59:46.027Z, Instrument: MS...",2015-07-27 08:59:46.027,2015-07-27 08:59:46.027,2015-07-27 08:59:46.027,2019-02-04 18:18:31.878,...,2015-07-27T08:59:42.000000Z,135849421.0,0.0000,PASSED,2015-07-27T08:59:42.000000Z,0.903994,-0.088293,23.694757,23.989523,gs://gcp-public-data-sentinel-2/tiles/34/N/HF/...


In [14]:
merged.columns

Index(['sentinel_id', 'title', 'link', 'link_alternative', 'link_icon',
       'summary', 'datatakesensingstart', 'beginposition', 'endposition',
       'ingestiondate', 'orbitnumber', 'relativeorbitnumber',
       'cloudcoverpercentage', 'gmlfootprint', 'format', 'instrumentshortname',
       'sensoroperationalmode', 'instrumentname', 'footprint', 's2datatakeid',
       'platformidentifier', 'orbitdirection', 'platformserialidentifier',
       'processingbaseline', 'processinglevel', 'producttype', 'platformname',
       'size', 'tileid', 'hv_order_tileid', 'filename', 'identifier', 'uuid',
       'level1cpdiidentifier', 'granuleidentifier', 'datastripidentifier',
       'index', 'GRANULE_ID', 'PRODUCT_ID', 'DATATAKE_IDENTIFIER', 'MGRS_TILE',
       'SENSING_TIME', 'TOTAL_SIZE', 'CLOUD_COVER', 'GEOMETRIC_QUALITY_FLAG',
       'GENERATION_TIME', 'NORTH_LAT', 'SOUTH_LAT', 'WEST_LON', 'EAST_LON',
       'BASE_URL'],
      dtype='object')

In [15]:
merged['index'].tolist()

[7725269,
 11241382,
 11273022,
 8764011,
 8231314,
 822721,
 3701925,
 1911926,
 5710807,
 8508100,
 12158332,
 2279478,
 805527,
 13025774,
 4054446,
 11016874,
 9316782,
 7790633,
 8452064,
 8166815,
 1410603,
 1483886,
 4477872,
 4543905,
 12628861]

In [20]:
rows_array = []

d = {}

for month in ['Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:
    print(f'Starting month {month}')
    month_num = months_dict[month][0]
    last_day = months_dict[month][1]
    
    api = get_api()
    products = get_products(api, footprint, date(2015, month_num, 1), date(2015, month_num, last_day))
    products_df = api.to_dataframe(products)
    products_df = products_df.reset_index()
    products_df = products_df.rename(columns={'index': 'sentinel_id'})
    products_df = products_df.drop_duplicates(subset=['tileid'])

    merged = products_df.merge(ee_index, left_on='title', right_on='PRODUCT_ID')
    rows = merged['index'].tolist()
    rows_array.append(rows)
    d[month] = len(rows)
    print(f'Ending month {month}')

Starting month Jul
Ending month Jul
Starting month Aug
Ending month Aug
Starting month Sep
Ending month Sep
Starting month Oct
Ending month Oct
Starting month Nov
Ending month Nov
Starting month Dec


Querying products: 100%|██████████████████████████████████████████████████████| 248/248 [00:07<00:00, 31.23 products/s]


Ending month Dec


In [21]:
d

{'Jul': 25, 'Aug': 26, 'Sep': 47, 'Oct': 8, 'Nov': 31, 'Dec': 126}

In [22]:
api = get_api()
products = get_products(api, footprint, date(2015, 7, 1), date(2015, 12, 31))
products_df = api.to_dataframe(products)
products_df = products_df.reset_index()
products_df = products_df.rename(columns={'index': 'sentinel_id'})
products_df = products_df.drop_duplicates(subset=['tileid'])

merged = products_df.merge(ee_index, left_on='title', right_on='PRODUCT_ID')
rows = merged['index'].tolist()

len(rows)

Querying products: 100%|██████████████████████████████████████████████████████| 418/418 [00:33<00:00, 12.66 products/s]


158

In [23]:
api = get_api()
products = get_products(api, footprint, date(2016, 1, 1), date(2016, 12, 31))
products_df = api.to_dataframe(products)
products_df = products_df.reset_index()
products_df = products_df.rename(columns={'index': 'sentinel_id'})
products_df = products_df.drop_duplicates(subset=['tileid'])

merged = products_df.merge(ee_index, left_on='title', right_on='PRODUCT_ID')
rows = merged['index'].tolist()

len(rows)

Querying products: 100%|████████████████████████████████████████████████████| 1864/1864 [05:04<00:00,  6.13 products/s]


187

In [11]:
get_tcis_for_year(2016, footprint, ee_index)

getting products


Querying products: 100%|████████████████████████████████████████████████████| 1864/1864 [04:46<00:00,  6.50 products/s]


finding rows
downloading tcis


In [7]:
get_tcis_for_year(2015, footprint, ee_index)

getting products


Querying products: 100%|██████████████████████████████████████████████████████| 418/418 [00:34<00:00, 12.26 products/s]


finding rows
downloading tcis
